In [5]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
#import mysql.connector
import plotly
import plotly.plotly as py #library https://plot.ly/pandas/
import plotly.graph_objs as go
#NASAQ 3 Year NIGR>25, preIPO 招股说明书上市前两年净收入。分析数字背后的故事，看懂数字背后的金融来去，包括投资到未来有成长空间的。软件和电商类优先。
plotly.tools.set_credentials_file(username='zhenxu66', api_key='yZ5rrla4FGut06wbewMo')

In [6]:
df_allndlist= pd.read_csv('./csv/list_nyseall.csv')

In [7]:
df_allndlist.head(2)

,id,symbol,name,sector,industry,country,board,prof_marginPCT,ppe,marketCapMillion,...,priceNow,recomm1buy5sell,SMA20PCT,SMA50PCT,SMA200PCT,relVolumn,2014to2013,2015to2014,2016to2015,2017to2016
0,1,AWP,Alpine Global Premier Properties Fund,Financial,Closed-End Fund - Foreign,USA,[NYSE],0.0,0.0,0.0,...,0.0554,0.0,0.0062,-0.0662,-0.1123,0.0072,0.0,0.0,0.0,0.0
1,2,AGD,Alpine Global Dynamic Dividend Fund,Financial,Closed-End Fund - Foreign,USA,[NYSE],0.0,0.0,0.0,...,0.0939,0.0,-0.0079,-0.0606,-0.1043,0.0137,0.0,0.0,0.0,0.0


In [8]:
df_allndlist_good = df_allndlist[(df_allndlist['SMA20PCT']>0.1)
                                     & (df_allndlist['SMA50PCT']>0.1) & (df_allndlist['SMA200PCT']>0.1)
                                 & (df_allndlist['2015to2014']>0) & (df_allndlist['2016to2015']>0)
                                & (df_allndlist['2017to2016']>0)]
print(df_allndlist_good.sort_values(by=['SMA200PCT']))

        id symbol                            name      sector  \
549    550    MPW  Medical Properties Trust, Inc.   Financial   
1007  1008   CIEN               Ciena Corporation  Technology   

                          industry country   board  prof_marginPCT    ppe  \
549   REIT - Healthcare Facilities     USA  [NYSE]           0.394  0.171   
1007       Communication Equipment     USA  [NYSE]           0.256  0.051   

      marketCapMillion     ...     priceNow recomm1buy5sell SMA20PCT  \
549             6020.0     ...       0.1633            0.03   0.1044   
1007            4880.0     ...       0.3404            0.02   0.1210   

      SMA50PCT SMA200PCT  relVolumn  2014to2013  2015to2014  2016to2015  \
549     0.1075    0.1895     0.0229     -0.4382      1.7920      0.6201   
1007    0.1180    0.2833     0.0147      0.5243      1.2871      5.2213   

      2017to2016  
549       0.2846  
1007     16.3861  

[2 rows x 41 columns]


In [9]:
df_allndlist_25 = df_allndlist[(df_allndlist['2015to2014']>0.25)
                                     & (df_allndlist['2016to2015']>0.25) & (df_allndlist['2017to2016']>0.25)]
print('Total stocks that satisfy four year continous NIGR>25%:'+str(len(df_allndlist_25))+ ' out of '
      +str(len(df_allndlist))+ ' total nasdaq stock that IPO before 2014'+'\n' )
if len(df_allndlist_25)<10:
    print(df_allndlist_25)
else:
    print(df_allndlist_25.name.tolist())
    print(df_allndlist_25.symbol.tolist())

Total stocks that satisfy four year continous NIGR>25%:56 out of 1283 total nasdaq stock that IPO before 2014

['ZTO Express (Cayman) Inc.', 'Western Alliance Bancorporation', 'WellCare Health Plans, Inc.', 'Walker & Dunlop, Inc.', 'Vivint Solar, Inc.', 'Veeva Systems Inc.', 'Vail Resorts, Inc.', 'US Foods Holding Corp.', 'TriNet Group, Inc.', 'TransUnion', 'Summit Materials, Inc.', 'Sprint Corporation', 'Sogou Inc.', 'Rosetta Stone Inc.', 'Rexford Industrial Realty, Inc.', 'ReneSola Ltd', 'Regis Corporation', 'Rayonier Advanced Materials Inc.', 'Post Holdings, Inc.', 'Physicians Realty Trust', 'Phillips 66 Partners LP', 'PennyMac Financial Services, Inc.', 'Paycom Software, Inc.', 'NexPoint Residential Trust, Inc.', 'Medical Properties Trust, Inc.', 'Mechel PAO', 'Martin Marietta Materials, Inc.', 'Mallinckrodt Public Limited Company', 'MagnaChip Semiconductor Corporation', 'Jupai Holdings Limited', 'Invesco Mortgage Capital Inc.', 'Grupo Supervielle S.A.', 'GrubHub Inc.', 'GoDaddy In

In [10]:
#df_allndlist_25.info()

In [11]:
df_sector = df_allndlist[df_allndlist['sector']=='Technology']
data_sector = [go.Histogram(
        x=df_allndlist['sector'],
        name='all',
        marker=dict(
            color='#FFD7E9',
        ),
        opacity=0.75
    ),
               go.Histogram(
        x=df_allndlist_25['sector'],
        name='NIGR>25',
        marker=dict(
            color='#EB89B5',
        ),
        opacity=0.75
    ),
           ]

layout = go.Layout(
    title='Sector hist Results',
    xaxis=dict(
        title='Sector',
    ),
    yaxis=dict(
        title='Count',
        automargin=True,
        type = 'log'
    ),
    bargap=0.2,
    bargroupgap=0.1,
    autosize=True

)
fig = go.Figure(data=data_sector, layout=layout)
py.iplot(fig, filename='styled histogram')

In [12]:
def plot_df_industry_symbols(df, sector):
    df_sector = df[df['sector']==sector]
    def industry_merge_company(industry):
        df_sector_industry = df_sector[df_sector['industry']==industry]
        return df_sector_industry['symbol'].str.cat(sep=',')
    
    value_counts = df_sector['industry'].value_counts()
    df_industry = value_counts.rename_axis('industry').reset_index(name='counts')
    df_industry['symbols'] = df_industry['industry'].apply(industry_merge_company)
    trace = go.Pie(labels=df_industry['industry'], values=df_industry['counts'], text = df_industry.symbols, textinfo='label+value')
    return trace

def df_industry_symbols(df, sector):
    df_sector = df[df['sector']==sector]
    def industry_merge_company(industry):
        df_sector_industry = df_sector[df_sector['industry']==industry]
        return df_sector_industry['symbol'].str.cat(sep=',')
    
    value_counts = df_sector['industry'].value_counts()
    df_industry = value_counts.rename_axis('industry').reset_index(name='counts')
    df_industry['symbols'] = df_industry['industry'].apply(industry_merge_company)
    return df_industry
trace_25 = plot_df_industry_symbols(df_allndlist,'Technology')
py.iplot([trace_25],filename='basic_pie_chart')

In [13]:
len(df_industry_symbols(df_allndlist,'Technology'))

26

In [14]:
#df_industry_symbols(df_allndlist_25,'Technology')
#df_industry_symbols(df_allndlist,'Technology')
#merge two plots together
fig = {
  "data": [
    {
      "values": df_industry_symbols(df_allndlist_25,'Technology')['counts'].tolist(),
      "labels": df_industry_symbols(df_allndlist_25,'Technology')['industry'].tolist(),
      "text":   df_industry_symbols(df_allndlist_25,'Technology').symbols.tolist(),
      "domain": {"x": [0, .48]},
      "name": "NIRG >25%",
      "hoverinfo":"label+percent+name+text",
      "hole": .4,
      "type": "pie"
    },
    {
      "values": df_industry_symbols(df_allndlist,'Technology')['counts'].tolist(),
      "labels": df_industry_symbols(df_allndlist,'Technology')['industry'].tolist(),
      "text":   df_industry_symbols(df_allndlist,'Technology').symbols.tolist(),
      "textposition":"none",
      "domain": {"x": [.52, 1]},
      "name": "All Nasdaq",
      "hoverinfo":"label+percent+name+text",
      "hole": .4,
      "type": "pie"
    }],
  "layout": {
        "title":"Technology Section Compare",
        "annotations": [
            {
                "font": {
                    "size": 20
                },
                "showarrow": False,
                "text": "NIRG >25%"+'\n'+str(len(df_industry_symbols(df_allndlist_25,'Technology'))),
                "x": 0.15,
                "y": 0.5
            },
            {
                "font": {
                    "size": 20
                },
                "showarrow": False,
                "text": "All Nasdaq"+'\n'+str(len(df_industry_symbols(df_allndlist,'Technology'))),
                "x": 0.85,
                "y": 0.5
            }
        ]
    }
}
py.iplot(fig, filename='donut')

C:\Users\zxu\AppData\Local\Continuum\Anaconda3\lib\site-packages\plotly\tools.py:1428: UserWarning:

Looks like you used a newline character: '\n'.

Plotly uses a subset of HTML escape characters
to do things like newline (<br>), bold (<b></b>),
italics (<i></i>), etc. Your newline characters 
have been converted to '<br>' so they will show 
up right on your Plotly figure!



In [15]:
def sector_plot(sector):
    fig = {
      "data": [
        {
          "values": df_industry_symbols(df_allndlist_25,sector)['counts'].tolist(),
          "labels": df_industry_symbols(df_allndlist_25,sector)['industry'].tolist(),
          "text":   df_industry_symbols(df_allndlist_25,sector).symbols.tolist(),
          "domain": {"x": [0, .48]},
          "name": "NIRG >25%",
          "hoverinfo":"label+percent+name+text",
          "hole": .4,
          "type": "pie"
        },
        {
          "values": df_industry_symbols(df_allndlist,sector)['counts'].tolist(),
          "labels": df_industry_symbols(df_allndlist,sector)['industry'].tolist(),
          "text":   df_industry_symbols(df_allndlist,sector).symbols.tolist(),
          "textposition":"none",
          "domain": {"x": [.52, 1]},
          "name": "All Nasdaq",
          "hoverinfo":"label+percent+name+text",
          "hole": .4,
          "type": "pie"
        }],
      "layout": {
            "title":"Services Section Compare",
            "annotations": [
                {
                    "font": {
                        "size": 20
                    },
                    "showarrow": False,
                    "text": "NIRG >25%"+'\n'+str(len(df_industry_symbols(df_allndlist_25,sector))),
                    "x": 0.15,
                    "y": 0.5
                },
                {
                    "font": {
                        "size": 20
                    },
                    "showarrow": False,
                    "text": "All Nasdaq"+'\n'+str(len(df_industry_symbols(df_allndlist,sector))),
                    "x": 0.85,
                    "y": 0.5
                }
            ]
        }
    }
    return fig
py.iplot(sector_plot('Services'), filename='donut')

C:\Users\zxu\AppData\Local\Continuum\Anaconda3\lib\site-packages\plotly\tools.py:1428: UserWarning:

Looks like you used a newline character: '\n'.

Plotly uses a subset of HTML escape characters
to do things like newline (<br>), bold (<b></b>),
italics (<i></i>), etc. Your newline characters 
have been converted to '<br>' so they will show 
up right on your Plotly figure!



In [16]:
data_ipo_year = [go.Histogram(
        x=df_allndlist['ipo_year'],
        name='all',
        marker=dict(
            color='#FFD7E9',
        ),
        opacity=0.75
    ),
               go.Histogram(
        x=df_allndlist_25['ipo_year'],
        name='NIGR>25',
        marker=dict(
            color='#EB89B5',
        ),
        opacity=0.75
    ),
           ]

layout = go.Layout(
    title='ipo_year hist Results',
    xaxis=dict(
        title='ipo_year',
    ),
    yaxis=dict(
        title='Count',
        automargin=True,
        type = 'log'
    ),
    bargap=0.2,
    bargroupgap=0.1,
    autosize=True

)
fig = go.Figure(data=data_ipo_year, layout=layout)
py.iplot(fig, filename='styled histogram')

In [17]:
data_country = [go.Histogram(
        x=df_allndlist['country'],
        name='all',
        marker=dict(
            color='#FFD7E9',
        ),
        opacity=0.75
    ),
               go.Histogram(
        x=df_allndlist_25['country'],
        name='NIGR>25',
        marker=dict(
            color='#EB89B5',
        ),
        opacity=0.75
    ),
           ]

layout = go.Layout(
    title='country hist Results',
    xaxis=dict(
        title='country',
    ),
    yaxis=dict(
        title='Count',
        automargin=True,
        type ='log'
    ),
    bargap=0.2,
    bargroupgap=0.1,
    autosize=True

)
fig = go.Figure(data=data_country, layout=layout)
py.iplot(fig, filename='styled histogram')

In [18]:
df_industry_sum = df_allndlist.groupby('industry').marketCapMillion.agg('sum')
print('Total # of industries: '+ str(len(df_industry_sum))+ '\n')
print(df_industry_sum.sort_values(ascending=False)[:30])

#df_allndlist_25.groupby('industry').marketCapMillion.agg('sum')

Total # of industries: 173

industry
Specialty Retail, Other                419934.77
Application Software                   406230.50
Credit Services                        301443.07
Life Insurance                         226570.00
Semiconductor - Integrated Circuits    206445.79
Asset Management                       183304.25
Chemicals - Major Diversified          176902.74
Oil & Gas Pipelines                    175984.63
Information Technology Services        165360.25
Drugs - Generic                        128690.00
Industrial Metals & Minerals           128247.07
Drug Manufacturers - Major             127720.00
Foreign Regional Banks                 126920.51
Business Services                      126806.37
Resorts & Casinos                      125430.67
Oil & Gas Drilling & Exploration       124903.60
Telecom Services - Domestic            118714.62
Air Delivery & Freight Services        107802.00
Electric Utilities                     106296.55
Auto Manufacturers - Major      

In [19]:
df_industry_25_sum = df_allndlist_25.groupby('industry').marketCapMillion.agg('sum')
print('Total # of industries: '+ str(len(df_industry_25_sum))+ '\n')
print(df_industry_25_sum.sort_values(ascending=False)[:30])

#df_allndlist_25.groupby('industry').marketCapMillion.agg('sum')

Total # of industries: 40

industry
Wireless Communications            25260.00
Diversified Computer Systems       17140.00
Business Services                  15770.00
General Building Materials         14233.65
Healthcare Information Services    13330.00
Health Care Plans                  13250.00
Air Delivery & Freight Services    13080.00
Internet Software & Services       12600.00
Discount, Variety Stores           11700.00
Internet Information Providers     11379.46
REIT - Industrial                  10730.00
Resorts & Casinos                  10440.00
Application Software               10052.59
REIT - Healthcare Facilities        9140.00
Aluminum                            7790.00
Paper & Paper Products              7400.00
Food Wholesale                      6930.00
Food - Major Diversified            6260.00
Oil & Gas Refining & Marketing      6110.00
Mortgage Investment                 6052.05
Packaging & Containers              5930.00
Regional - Pacific Banks            5110

In [20]:
fig = {
    'data': [
        {
            'x': df_allndlist_25['ppe'], 
            'y': df_allndlist_25['marketCapMillion'], 
            'text': df_allndlist_25.symbol + ' name:'+ df_allndlist_25.name,  #cannot access columns with '' '' 
            'mode': 'markers', #+text
            'marker':dict(
                size= 8, #30*df_allnd100list['inst_own'],
                color = df_allndlist_25['recomm1buy5sell'], #set color equal to a variable
                colorscale='Viridis',
                showscale=True
            ),
            'name': 'Results'},      

    ],
    'layout': {
        'title': 'market_volumn compared with price per earnings and profit_margin (%)',
        'xaxis': {'title': 'price per earnings','autorange':True,'type': 'log'},
        'yaxis': {'title': "mark_volumn",'autorange':True,'type': 'log'},
        'showlegend': True,
        'annotations':[ #not working
            dict(
                x=1100,#df_allnd100list[df_allnd100list.symbol=='AAPL']['mark_cap(billion)'].item(),
                y=4800,#df_allnd100list[df_allnd100list.symbol=='AAPL']['shares_out(million)'].item(),
                xref='x',
                yref='y',
                text='dict Text is here',
                showarrow=True,
                arrowhead=7,
                ax=0,
                ay=-40
            ),
        ]
    }
}

In [21]:
py.iplot(fig, filename='scatter')  #py.plot will generate a webpage

In [22]:
column_PCT=[]
for i in df_allndlist_25.columns.tolist():
    if i[-3:]=='PCT':
        column_PCT.append(i)
print(df_allndlist_25.columns.tolist())
print(column_PCT)


['id', 'symbol', 'name', 'sector', 'industry', 'country', 'board', 'prof_marginPCT', 'ppe', 'marketCapMillion', 'descShort', 'url_reuter', 'url_marketwatch', 'ipo_year', 'url', 'EPS', 'insideOwnPCT', 'forwardPPE', 'nextYEPS', 'insideOwnTrans6monthPCT', 'instOwnPCT', 'thisYearGrowthEPSPCT', 'instOwnTrans3monthPCT', 'nextYearGrowthEPSPCT', 'ROAPCT', 'ROEPCT', 'dividAnn', 'ROIPCT', 'dividYieldAnnPCT', 'dividPayoutRatioPCT', 'avgVolumn3MonthMillion', 'priceNow', 'recomm1buy5sell', 'SMA20PCT', 'SMA50PCT', 'SMA200PCT', 'relVolumn', '2014to2013', '2015to2014', '2016to2015', '2017to2016']
['prof_marginPCT', 'insideOwnPCT', 'insideOwnTrans6monthPCT', 'instOwnPCT', 'thisYearGrowthEPSPCT', 'instOwnTrans3monthPCT', 'nextYearGrowthEPSPCT', 'ROAPCT', 'ROEPCT', 'ROIPCT', 'dividYieldAnnPCT', 'dividPayoutRatioPCT', 'SMA20PCT', 'SMA50PCT', 'SMA200PCT']


In [23]:
group = df_allndlist_25.columns.to_series().groupby(df_allndlist_25.dtypes).groups
dict_group = {k.name: v for k, v in group.items()}
float_column = dict_group.get('float64').tolist()
print(float_column)

['prof_marginPCT', 'ppe', 'marketCapMillion', 'EPS', 'insideOwnPCT', 'forwardPPE', 'nextYEPS', 'insideOwnTrans6monthPCT', 'instOwnPCT', 'thisYearGrowthEPSPCT', 'instOwnTrans3monthPCT', 'nextYearGrowthEPSPCT', 'ROAPCT', 'ROEPCT', 'dividAnn', 'ROIPCT', 'dividYieldAnnPCT', 'dividPayoutRatioPCT', 'avgVolumn3MonthMillion', 'priceNow', 'recomm1buy5sell', 'SMA20PCT', 'SMA50PCT', 'SMA200PCT', 'relVolumn', '2014to2013', '2015to2014', '2016to2015', '2017to2016']


In [24]:
df_allndlist_25_float= df_allndlist_25[float_column]
df_allndlist_25_float.corr()['priceNow'].sort_values(ascending = False)

priceNow                   1.000000
nextYEPS                   0.828604
marketCapMillion           0.574053
EPS                        0.504525
dividAnn                   0.354914
ppe                        0.274769
SMA200PCT                  0.189820
ROIPCT                     0.154654
2014to2013                 0.125684
ROAPCT                     0.124230
ROEPCT                     0.116366
forwardPPE                 0.072897
instOwnPCT                 0.011437
instOwnTrans3monthPCT      0.011387
relVolumn                 -0.004085
nextYearGrowthEPSPCT      -0.033090
recomm1buy5sell           -0.035372
thisYearGrowthEPSPCT      -0.043944
2016to2015                -0.053245
dividPayoutRatioPCT       -0.062907
SMA20PCT                  -0.071095
avgVolumn3MonthMillion    -0.077171
SMA50PCT                  -0.094723
prof_marginPCT            -0.105485
2017to2016                -0.135566
insideOwnPCT              -0.146934
dividYieldAnnPCT          -0.149166
2015to2014                -0

In [25]:
df_allndlist_25_PCT= df_allndlist_25[column_PCT]

In [26]:
#df_allndlist_25_PCT.corr()

In [27]:
df_allndlist_25_PCT.corr()['SMA20PCT'].sort_values(ascending = False)

SMA20PCT                   1.000000
SMA50PCT                   0.769581
ROAPCT                     0.364736
ROIPCT                     0.275794
prof_marginPCT             0.210393
nextYearGrowthEPSPCT       0.182171
dividYieldAnnPCT           0.119912
thisYearGrowthEPSPCT       0.085216
SMA200PCT                  0.083009
dividPayoutRatioPCT       -0.008114
insideOwnTrans6monthPCT   -0.053142
instOwnPCT                -0.053720
ROEPCT                    -0.087247
instOwnTrans3monthPCT     -0.088584
insideOwnPCT              -0.229444
Name: SMA20PCT, dtype: float64

In [28]:
#from pandas.plotting import scatter_matrix
#scatter_matrix(df_allndlist_25_PCT,figsize= (16,12),alpha=0.3)

In [29]:
data_industry = [go.Histogram(
        y=df_allndlist_25['industry'])
           ]

layout = go.Layout(
    title='industry hist Results',
    xaxis=dict(
        title='Count',
        type ='log'
    ),
    yaxis=dict(
        title='industry',
        automargin=True
    ),
    bargap=0.2,
    autosize=False,
    width=1000,
    height=2000,
    margin=go.layout.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),

)
fig = go.Figure(data=data_industry, layout=layout)
py.iplot(fig, filename='styled histogram')

In [30]:
data_industry = [go.Histogram(
        y=df_allndlist['industry'])
           ]

layout = go.Layout(
    title='industry hist Results',
    xaxis=dict(
        title='Count',
        type ='log'
    ),
    yaxis=dict(
        title='industry',
        automargin=True
    ),
    bargap=0.2,
    autosize=False,
    width=1000,
    height=2000,
    margin=go.layout.Margin(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),

)
fig = go.Figure(data=data_industry, layout=layout)
py.iplot(fig, filename='styled histogram')